In [48]:
from sqlalchemy import create_engine ##to inRWExr with postgre sql
import pyodbc ## to query the sql server
import pandas as pd ## to data extra and data load
import os ##  
import requests
import mysql.connector
from mysql.connector import Error



pwd = "demopass"
uid = "etl"
server = "localhost"

## extract data from MySQL
def extract():
    try:
        connection = mysql.connector.connect(host='localhost', 
                                            database='classicmodels',
                                            user=uid,
                                            password=pwd)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to the database: ", record)

            mycursor =  connection.cursor()

            mycursor.execute("""SELECT * from customers""")

            myresult = mycursor.fetchall()

            
            df = pd.read_sql_query(f'select * FROM customers', connection)
            load(df, myresult)
    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

## load data to postgres
def load(df, tbl):
    try:
        rows_imported = 0
        engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:5432/Adventure')
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
        #save df to postgres
        df.to_sql('fstg_{tbl}', engine, if_exists='replace', index=False)
        rows_imported += len(df)
        #add elapser time to final print out
        print("Data imported sucessful")
    except Error as e:
        print("Data load error: " + str(e))

try:
    #call extract function
    extract()
except Exception as e:
    print("Error while extracting data: " + str(e))

Connected to MySQL Server version 8.1.0
You're connected to the database:  ('classicmodels',)
importing rows 0 to 122... for table [(103, 'Atelier graphique', 'Schmitt', 'Carine ', '40.32.2555', '54, rue Royale', None, 'Nantes', None, '44000', 'France', 1370, Decimal('21000.00')), (112, 'Signal Gift Stores', 'King', 'Jean', '7025551838', '8489 Strong St.', None, 'Las Vegas', 'NV', '83030', 'USA', 1166, Decimal('71800.00')), (114, 'Australian Collectors, Co.', 'Ferguson', 'Peter', '03 9520 4555', '636 St Kilda Road', 'Level 3', 'Melbourne', 'Victoria', '3004', 'Australia', 1611, Decimal('117300.00')), (119, 'La Rochelle Gifts', 'Labrune', 'Janine ', '40.67.8555', '67, rue des Cinquante Otages', None, 'Nantes', None, '44000', 'France', 1370, Decimal('118200.00')), (121, 'Baane Mini Imports', 'Bergulfsen', 'Jonas ', '07-98 9555', 'Erling Skakkes gate 78', None, 'Stavern', None, '4110', 'Norway', 1504, Decimal('81700.00')), (124, 'Mini Gifts Distributors Ltd.', 'Nelson', 'Susan', '41555514

/var/folders/qf/z_6nd3fj27n0qhtxf_q9fkv80000gn/T/ipykernel_35028/4100881154.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'select * FROM customers', connection)


SyntaxError: invalid syntax (3412109671.py, line 13)